# Computer Vision Learner

[`vision.learner`](/vision.learner.html#vision.learner) is the module that defines the `Conv_Learner` class, to easily get a model suitable for transfer learning.

In [ ]:
from fastai.gen_doc.nbdoc import *
from fastai.vision import *
from fastai import *

## Transfer learning

Transfer learning is a technique where you use a model trained on a very large dataset (usually [ImageNet](http://image-net.org/) in computer vision) and then adapt it to your own dataset. The idea is that it has learned to recognize many features on all of this data, and that you will benefit from this knowledge, especially if your dataset is small, compared to starting from a randomly initiliazed model. It has been proved in [this article](https://arxiv.org/abs/1805.08974) on a wide range of tasks that transfer learning nearly always give better results.

In practice, you need to change the last part of your model to be adapted to your own number of classes. Most convolutional models end with a few linear layers (a part will call head). The last convolutional layer will have analyzed features in the image that went through the model, and the job of the head is to convert those in predictions for each of our classes. In transfer learning we will keep all the convolutional layers (called the body or the backbone of the model) with their weights pretrained on ImageNet but will define a new head initiliazed randomly.

Then we will train the model we obtain in two phases: first we freeze the body weights and only train the head (to convert those analyzed features into predictions for our own data), then we unfreeze the layers of the backbone (gradually if necessary) and fine-tune the whole model (possily using differential learning rates).

The [`ConvLearner`](/vision.learner.html#ConvLearner) class helps you to automatically get a pretrained model from a given architecture with a custom head that is suitable for your data.

In [ ]:
show_doc(ConvLearner, doc_string=False)

This class creates a [`Learner`](/basic_train.html#Learner) object from the [`data`](/vision.data.html#vision.data) object and model inferred from it with the backbone given in `arch`. Specifically, it will cut the model defined by `arch` (randomly initialized if `pretrained` is False) at the last convolutional layer by default (or as defined in `cut`, see below) and add:
- an [`AdaptiveConcatPool2d`](/layers.html#AdaptiveConcatPool2d) layer,
- a [`Flatten`](/layers.html#Flatten) layer,
- blocks of \[[`nn.BatchNorm1d`](https://pytorch.org/docs/stable/nn#torch.nn.BatchNorm1d), [`nn.Dropout`](https://pytorch.org/docs/stable/nn#torch.nn.Dropout), [`nn.Linear`](https://pytorch.org/docs/stable/nn#torch.nn.Linear), [`nn.ReLU`](https://pytorch.org/docs/stable/nn#torch.nn.ReLU)\] layers.

The blocks are defined by the `lin_ftrs` and `ps` arguments. Specifically, the first block will have a number of inputs inferred from the backbone `arch` and the last one will have a number of outputs equal to `data.c` (which contains the number of classes of the data) and the intermediate blocks have a number of inputs/outputs determined by `lin_frs` (of course a block has a number of inputs equal to the number of outputs of the previous block). The default is to have an intermediate hidden size of 512 (which makes two blocks `model_activation` -> 512 -> `n_classes`). If you pass a float then the final dropout layer will have the value `ps`, and the remaining will be `ps/2`. If you pass a list then the values are used for dropout probabilities directly.

Note that the very last block doesn't have a [`nn.ReLU`](https://pytorch.org/docs/stable/nn#torch.nn.ReLU) activation, to allow you to use any final activation you want (generally included in the loss function in pytorch). Also, the backbone will be frozen if you choose `pretrained=True` (so only the head will train if you call [`fit`](/basic_train.html#fit)) so that you can immediately start phase one of training as described above.

Alternatively, you can define your own `custom_head` to put on top of the backbone. If you want to specify where to split `arch` you should so in the argument `cut` which can either be the index of a specific layer (the result will not include that layer) or a function that, when passed the model, will return the backbone you want.

The final model obtained by stacking the backbone and the head (custom or defined as we saw) is then separated in groups for gradual unfreezeing or differential learning rates. You can specify of to split the backbone in groups with the optional argument `split_on` (should be a function that returns those groups when given the backbone).  

The `kwargs` will be passed on to [`Learner`](/basic_train.html#Learner), so you can put here anything that [`Learner`](/basic_train.html#Learner) will accept ([`metrics`](/metrics.html#metrics), `loss_func`, `opt_func`...)

In [ ]:
data = URLs.get_mnist()

In [ ]:
learner = ConvLearner(data, models.resnet18, metrics=[accuracy])
learner.fit_one_cycle(1,1e-3)

### Customize your model

You can customize [`ConvLearner`](/vision.learner.html#ConvLearner) for your own models default `cut` and `split_on` functions by adding it them the dictionary `model_meta`. The key should be your model and the value should be a dictionary with the keys `cut` and `split_on` (see the source code for examples). The constructor will call [`create_body`](/vision.learner.html#create_body) and [`create_head`](/vision.learner.html#create_head) for you based on `cut`; you can also call them yourself, which is particularly useful for testing.

In [ ]:
show_doc(create_body)

In [ ]:
show_doc(create_head, doc_string=False)

Model head that takes `nf` features, runs through `lin_ftrs`, and ends with `nc` classes. `ps` is the probability of the dropouts, as documented above in [`ConvLearner`](/vision.learner.html#ConvLearner).

### Utility methods

In [ ]:
show_doc(num_features)

In [ ]:
show_doc(ClassificationInterpretation)

This provides a confusion matrix and visualization of the most incorrect images. Pass in your [`data`](/vision.data.html#vision.data), calculated `preds`, actual `y`, and the class of your loss function, and then use the methods below to view the model interpretation results. For instance:

In [ ]:
learn = ConvLearner(URLs.get_mnist(), models.resnet18)
learn.fit(1)
preds,y = learn.get_preds()
interp = ClassificationInterpretation(data, preds, y, loss_class=nn.CrossEntropyLoss)

In [ ]:
show_doc(ClassificationInterpretation.plot_top_losses)

The `k` items are arranged as a square, so it will look best if `k` is a square number (4, 9, 16, etc). The title of each image shows: prediction, actual, loss, probability of actual class.

In [ ]:
interp.plot_top_losses(9, figsize=(7,7))

In [ ]:
show_doc(ClassificationInterpretation.top_losses)

Returns tuple of *(losses,indices)*.

In [ ]:
interp.top_losses(9)

In [ ]:
show_doc(ClassificationInterpretation.plot_confusion_matrix)

In [ ]:
interp.plot_confusion_matrix()

In [ ]:
show_doc(ClassificationInterpretation.confusion_matrix)

In [ ]:
interp.confusion_matrix()

## Undocumented Methods - Methods moved below this line will intentionally be hidden

In [ ]:
show_doc(ClassificationInterpretation.from_learner)

## New Methods - Please document or move to the undocumented section

In [ ]:
show_doc(ClassificationInterpretation.most_confused)